# Tutorial 2: Surrogate modelling with Kriging

A major issue in Uncertainty Quantification is the high-cost of evaluating the computational model one time (e.g. a CFD code) - combined with the large number of evaluations needed.  *Surrogate modelling* is a general term, describing replacing the CFD code with a cheap map that approximates it.  The polynomial stochastic methods are also surrogate models when used for interpolation (as opposed to quadrature).

With $d \in\mathbb{N}$ the number of parameters (the *dimension*), and a scalar output, any simulation code $g(\cdot)$ is schematically:
$$
g: \mathbb{R}^d\rightarrow \mathbb{R}
$$
to be approximated by a surrogate model $\hat g(\cdot)$:
$$
g(\mathbf{x}) \simeq \hat g(\mathbf{x}),\quad \mathbf{x}\in\Omega \subset \mathbb{R}^d.
$$

Surrogate models can be interpolative or regressive.  Some examples:

* Stochastic polynomial interpolants, including sparse-grids;
* Linear regression;
* Radial-basis function interpolation.

Our goal in this tutorial is to implement and explore the *stochastic* regressive surrogate model $\hat g$ called variously "Kriging" (by engineers) or "Gaussian Process Regression" (by mathematicians).  For simplicity will will consider the case $d=1$, but the generalization to higher-dimensions is striaghtforward.

We will consider the test-function
$$
g(x) = \sin(cx^2)\cdot (1+x) + (1+x^2),\quad c=40,
$$

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.interpolate as interpolate
import scipy.stats as stats

c = 40
def g(x): return np.sin(x*x*c)*(1+x) + (1+x**2)

In [ ]:
x = np.linspace(0, 1, 201)
plt.plot(x, g(x), '-')
plt.xlabel(r'$x$'); plt.ylabel(r'$g(\cdot)$')

## Discretizing the domain $\Omega$
**EXERCISE 1**

Generate about $M=10$ *training nodes* randomly on $\Omega = [0,1]$ (e.g. with `np.random.rand()`), and additionally about $N=100$ *test nodes* uniformly spaced on $\Omega = [0,1]$.  Concatenate these arrays (e.g. with `np.hstack()`) to get a single array of both training and test nodes, "all".  To check your code `x_all.shape == (M+N,)` should be `True`.

In [ ]:
M =
N =

x_train = 
x_test = 
x_all = 

## Sampling from the prior

The first step is computing the prior covariance matrix based on the definition in the notes:
$$
[P]_{ij} := \sigma_y^2 r(x_i, x_j, l),\quad r(x_i,x_j,l) := \exp\left\{-\frac{|x_i-x_j|^2}{2 l^2}\right\}
$$
In the following code `r()` with array arguments `xi` and `xj`, computes the function $r()$ above, for all combinations of array entries, and returns the result in a matrix. 

In [ ]:
def distances_all(xi,xj): 
    return np.abs(xi[np.newaxis,:] - xj[:,np.newaxis])

def r(xi, xj, sigma_0, l): 
    return sigma_0**2 * np.exp(-distances_all(xi,xj)**2/ l**2)

**EXERCISE 2**

Consider the Gaussian process $\mathcal{Y}_0$ defined by some mean function $\mu(x)$ and $r()$, written:
$$
\mathcal{Y}_0 \sim \mathcal{GP}(\mu, r).
$$
We know from the lectures that by selecting a finite vector of nodes $\mathbf{x} = (x_1,\dots,x_N)$, we can define a multivariate random variable $\mathbf{Y}_0$:
$$
\mathbf{Y}_0 = \mathcal{Y}(\mathbf{x}) \sim \mathcal{N}(\mu(\mathbf{x}), P)
$$
Define a function `mu(x)`, and using this and the previously defined `r()`, generate random samples from $\mathbf{Y}_0$ for your *test nodes* only.

[Note: Use `scipy.stats.multivariate_normal()` with the parmeter `allow_singular=True` to surpress errors and warnings.  This is a numerical conditioning issue, due to floating-point accuracy - and does not result in any problems in this notebook.  Generating (e.g. 10) samples can be done with `scipy.stats.multivariate_normal(mu,P).rvs(10)`]

Plot a few (e.g. 10-20) samples of $\mathbf{Y}_0$ (each sample is a function).  Explore the effects of $\mu$, $\sigma_0$ and $l$ on the functions generated.  Do you think one of the *class* of functions generated would be a good fit for $g$?  Find parameters that you think represent a reasonable *prior* for $g()$.



## Adding data (likelihood)

In our Bayesian formulation the random-variable $\mathbf{Y} | \mathbf{d}$ (that we are trying to identify), is defined at both training- and test nodes.  Data $\mathbf{d}\in \mathbb{R}^M$ is only available at training nodes.  We need therefore:
 
1. An observation matrix $H \in \mathbb{R}^{M\times (M+N)}$, which extracts only the observed (training) values from the combined training + test vector $\mathbf{y}$.
2. The observation error matrix $R = \sigma_\epsilon^2 I\in \mathbb{R}^{M\times M}$ - whereby we assume Gaussian uncorrellated observation errors.  We assume throughout that $\sigma_\epsilon=1\times 10^{-3}$.

**EXERCISE 3**

Implement 1: There is a useful trick here: Numpy *slices*.  With the syntax `x_all[:M]` you extract the 1st `M` entries of `x_all`.  Similarly `x_all[M:]` returns all except the 1st `M` entries.  Apply this idea to an identity matrix of size $M+N$ to get a suitable matrix $H$.

Implement 2. As well as $R$, also evaluate the data $\mathbf{d} := g(\mathbf{x}_\mathrm{train}) + \epsilon$.  I.e. explicitly add a small about of noise to the observations to simulate a real measurement.  Use `stats.multivariate_normal(0*x_train, R).rvs()` to generate the noise.

**EXERCISE 4**

Now implementing Kriging is just a question of computing the posterior pdf from Bayes rule:
$$
p( \mathbf{y} \mid \mathbf{d}) \propto p(\mathbf{d}\mid  \mathbf{y}) \cdot p_0(\mathbf{y}).
$$
Since prior and likelihood are both Gaussian, the posterior is Gaussian
$$
\mathbf{Y} \mid \mathbf{d} \sim \mathcal{N}(\hat\mu, \hat\Sigma),
$$
and we need only find the mean and covariance matrix - which is just linear algebra:
$$
\hat\mu = \mu_y + K (\mathbf{d} - H\mu_y) \\
\hat\Sigma = (I - K H) P \\
K = PH^T (R+HPH^T)^{-1}.
$$
Beware that in this formula, the prior mean $\mu_0$ and covariance $P$ should be evaluated at *all* nodes.

Plot the mean, and a few random samples from the posterior.  Compare with the true function.  [For plotting using only the test nodes is convenient.]  You should see a good fit at the training nodes, and closeby - especially where training nodes are clustered.  The fit elsewhere may be poor.  Choose reasonable values for $\mu()$ and $\sigma_0$ given what you know about the function - adjust $l$ to try to get a reasonable fit (it may be difficult without more than 10 samples!).

**EXERCISE 5**

Take your code so far, and (without much modification), put in a form which can be called in the following way.  Check results are similar to your previous code.

In [ ]:
def kriging(M, N, x, d, mu, sigma_0, sigma_epsilon, l):
    """
    Kriging in 1-dimension for a single variable - following the
    Bayesian derivation and notation.
    
    Arguments:
      M,N [integers]: Number of training-, test-nodes.
      x [array (M+N)]: Node locations (both observations and predictions)
      d [array (M)]: Nodal observations ("data")
      mu [function]: Mean of prior.
      sigma_0 [float]: Standard-deviation of the prior.
      sigma_epsilon [float]: Standard-deviation of observation error.
      l [float]: Correlation length.
    Return:
      muhat [array (M+N)]: Posterior mean
      Sigmahat [array (M+N,M+N)]: Posterior covariance.
    """
    pass  # TODO
    

## Convergence study
**EXERCISE 6**

The most basic thing we require from numerical methods is *convergence*.  The following code will test the convergence of your `kriging()` function.  For an error measure the RMSE w.r.t. the kriging mean, at the test nodes is used:
$$
\epsilon := \sqrt{\frac{1}{N}\sum_{i=1}^N (g(\mathbf{x}_i) - \hat\mu(\mathbf{x}_i))^2}
$$
Both training and test-nodes are uniformly spaced to reduce noise.  You can see the variability in convergence by running `convergence_study()` multiple times in the cell below, which will add more lines to the same graph.

Which of the 3 parameters $\mu$, $\sigma_0$, $l$ is the most important for the convergence rate and absolute error?  What happens to the converenge for large values of $M$?  It should approach a constant *rate*? (I.e. straight-line in log-log plot.)  Is this rate familiar?  Why is does this rate result eventually?

In [ ]:
# Convergence study
def mu(x): ???   # TODO pick a reasonable prior
sigma_0 = ???
l = ???
sigma_epsilon = 1.e-3

def convergence_study(mu, sigma_0, sigma_epsilon, l):
    N = 1000
    Ms = 2**np.arange(2,12,dtype=np.int)
    err_M = np.zeros(len(Ms))
    for j,M in enumerate(Ms):
        x = np.hstack((np.linspace(0,1,M), np.linspace(0,1,N)))
        d = g(x[:M]) + stats.multivariate_normal(np.zeros(M), np.identity(M)*sigma_epsilon**2).rvs()
        muhat,_ = kriging(M, N, x, d, mu, sigma_0, sigma_epsilon, l)
        err_M[j] += np.sqrt(np.sum((g(x[M:]) - muhat)**2)/N)

    plt.loglog(Ms, err_M, 'x-')

    plt.xlabel(r'$M$')
    plt.ylabel(r'$\epsilon$')
    slope, intercept, r_value, p_value, std_err = \
        stats.linregress(np.log10(Ms)[-3:], np.log10(err_M)[-3:])
    print("Convergence rate = %.4g" % -slope)
    plt.loglog(Ms, 10**(intercept + slope*np.log10(Ms)), '--', color='0.5')   
    
convergence_study(mu, sigma_0, sigma_epsilon, l/4)
convergence_study(mu, sigma_0, sigma_epsilon, l/2)
convergence_study(mu, sigma_0, sigma_epsilon, l)
convergence_study(mu, sigma_0, sigma_epsilon, 2*l)

